## 문제 정의

YOLOv5를 사용한 색상, 흠집에 대한 이미지 학습으로 등급을 정확하게 분류하기에는 한계가 있음
실제로 무게와 당도가 등급 분류에 영향을 미침
특히 특급, 상급은 비슷한 이미지가 많아 정확성이 30%대로 매우 낮은 수치에 해당함
따라서 무게를 추론 해낼 수 있는 넓이, 높이와 같은 크기 수치와 등급의 상관 관계 분석을 실시

<br>

- 귀무 가설(Null Hypothesis) : 크기와 넓이가 특급, 상급 등급 분류에 영향을 주지 않는다.

- 대안 가설(Alternative Hypothesis) : 크기와 넓이가 특급, 상급 등급 분류에 영향을 준다.

## 데이터 전처리 (Data Preprocessing)

In [4]:
# 사용 라이브러리 임포트

import numpy as np
import pandas as pd
from random import sample

In [8]:
# 데이터 프레임 로드
df = pd.read_csv("data/dataset.csv")
df.head()
# High는 특급, Medium은 상급, Low는 보통급에 해당함

,ImageID,LabelName,Width,Height
0,apple_fuji_L_1,high,9.7,9.0
1,apple_fuji_L_10,high,9.6,8.9
2,apple_fuji_L_11,high,9.4,8.7
3,apple_fuji_L_12,high,9.7,8.7
4,apple_fuji_L_13,high,9.7,9.4


In [ ]:
# 특급과 상급을 비교하기 위해 보통은 데이터프레임으로부터 제외
filtered = # 여기에 슬라이싱 사용해서 high랑 medium 남게 할것

## 탐색적 데이터 분석 (EDA)

### 데이터 분석 (Data analysis)

In [ ]:
# 특급과 상급 크기, 넓이 평균값 비교

### 시각화(Visualization)

In [ ]:
# 등급에 따른 Height 분포 (히스토그램 사용)

In [ ]:
# 등급에 따른 width 분포 (히스토그램 사용)

### 순열검정법 (Permutation test)

##### 전체 순열 검정법 (exhaustive permutation test)
- 만약 히스토그램의 분포도가 정규 분포에 해당하면 실시
##### 부트스트래핑 (Bootstrapping)
- 만약 히스토그램의 분포도가 정규 분포에 해당하지 않으면 실시

##### 두 방법의 차이점
- 전체 순열 검정은 재표본추출(샘플링) 할때 중복 되지 않도록 한다. 했던 것을 다시 뽑으면 안됨
- 부트스트래핑은 재표본추출(샘플링) 할때 중복 될 수도 있도록 한다. 했던 것을 다시 뽑아도 됨


#### 크기(Height) 테스트

In [ ]:
obs = # 관측치 계산(observation) - 특급의 평균 크기

In [9]:
# 랜덤 셔플링 코드

In [10]:
# p-value 계산

In [ ]:
# 시각화

#### 넓이(Width) 테스트

In [ ]:
obs = # 관측치 계산(observation) - 특급의 평균 크기

In [ ]:
# 랜덤 셔플링 코드

In [ ]:
# p-value 계산

In [ ]:
# 시각화